In [ ]:
!pip install torch transformers bitsandbytes accelerate peft vllm

In [ ]:
pip install openpyxl

In [ ]:
pip install openai

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import pandas as pd
from openai import OpenAI
import json
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
API_KEY=""""""
client = OpenAI(
    api_key=API_KEY,
)
MAX_RETRIES = 3
SLEEP_DURATION = 5

In [ ]:
os.environ['VLLM_ALLOW_LONG_MAX_MODEL_LEN'] = '1'
model_id = "microsoft/Phi-3-mini-4k-instruct"
llm = LLM(model=model_id, max_model_len=16240)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

WARNING 08-22 00:46:40 config.py:1567] User-specified max_model_len (16240) is greater than the derived max_model_len (max_position_embeddings=4096 or model_max_length=None in model's config.json). This may lead to incorrect model outputs or CUDA errors. Make sure the value is correct and within the model context size.
INFO 08-22 00:46:40 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='microsoft/Phi-3-mini-4k-instruct', speculative_config=None, tokenizer='microsoft/Phi-3-mini-4k-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=16240, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 08-22 00:47:13 model_runner.py:732] Loading model weights took 7.1183 GB
INFO 08-22 00:47:16 gpu_executor.py:102] # GPU blocks: 1982, # CPU blocks: 682
INFO 08-22 00:47:18 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-22 00:47:18 model_runner.py:1028] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 08-22 00:47:37 model_runner.py:1225] Graph capturing finished in 18 secs.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

NameError: name 'model_id' is not defined

In [ ]:
def chatGPT4oresponse(ques):

    for _ in range(MAX_RETRIES):
        try:
            response = client.chat.completions.create(
                model='gpt-4o',
                messages=[
                    {"role": "user", "content": "{}".format(ques)},
                ],
                temperature=0,
            )
            print(response.choices[0].message.content)
            return(response.choices[0].message.content)
        except Exception as e:
            if _ == MAX_RETRIES - 1:  # Last iteration
                error = "An error occurred: {}\n".format(e)
                return error
            time.sleep(SLEEP_DURATION)  # Sleep before retrying

def base_gpt4_model(ques):
    for _ in range(MAX_RETRIES):
        try:
            response = client.chat.completions.create(
                model='gpt-4-turbo',
                messages=[
                    {"role": "user", "content": "{}".format(ques)},
                ],
                temperature=0,
            )
            return(response.choices[0].message.content)
        except Exception as e:
            if _ == MAX_RETRIES - 1:  # Last iteration
                return "An error occurred: {}".format(e)
            time.sleep(SLEEP_DURATION)  # Sleep before retrying

def generate_terraform_code(user_query,cloud_provider):
    instruction = "Generate Terraform code based on the provided requirements."
    prompt="""You are a Terraform consultant with expertise in AWS, Azure, and GCP. Your task is to develop a Terraform configuration based on the provided user query for the specified cloud provider.

    - *User Query:* {}
    - *Cloud Provider:* {}

    This Terraform configuration must include modules for essential components and design the modules to be reusable and easy to maintain.
    Adhere to the industry best practices for security and efficiency throughout the configuration. The output Terraform config should be in a structured format that is easy to understand and implement."""
    updated_prompt=prompt.format(user_query,cloud_provider)
    chat = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": updated_prompt}
    ]

    formatted_prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

    return formatted_prompt

In [ ]:
df=pd.read_excel('/content/Research_Analysis_Base_data_filtered_new.xlsx')
del df['Unnamed: 0']
print(len(df))

461


In [ ]:
df.head()

,Category,Cloud_provider,user_query,Terraform_Code
0,GoogleCloudPlatform,GCP,Please generate a Terraform configuration for ...,#\n# Copyright 2020 Google LLC\n#\n# Licensed ...
1,immutability-io,AWS,Please generate a Terraform configuration for ...,# DHCP Options\n\n## Set Terraform version con...
2,forseti-security,GCP,Please generate a Terraform configuration for ...,/**\n * Copyright 2019 Google LLC\n *\n * Lice...
3,cncf,AWS,Please generate a Terraform configuration for ...,"# resource ""azurerm_network_security_group"" ""c..."
4,terraform-google-modules,GCP,Please generate a Terraform configuration for ...,"resource ""google_compute_ssl_certificate"" ""def..."


In [ ]:
#df=df.head(10)

In [ ]:
prompt_lst=[]
for index, row in df.iterrows():
    user_query=row['user_query']
    cloud_provider=row['Cloud_provider']
    #print("user query is {}".format(user_query))
    try:
        updated_prompt=generate_terraform_code(user_query,cloud_provider)
        prompt_lst.append(updated_prompt)
    except Exception as error:
        print(error)

In [ ]:
prompt_lst[0]

'<|system|>\nGenerate Terraform code based on the provided requirements.<|end|>\n<|user|>\nYou are a Terraform consultant with expertise in AWS, Azure, and GCP. Your task is to develop a Terraform configuration based on the provided user query for the specified cloud provider.\n\n    - *User Query:* Please generate a Terraform configuration for setting up a Google Cloud Platform (GCP) project with necessary services, a VPC network, subnets, firewall rules, a NAT router, compute instances, instance templates, health checks, an instance group manager, and a global load balancer.\n    - *Cloud Provider:* GCP\n\n    This Terraform configuration must include modules for essential components and design the modules to be reusable and easy to maintain.\n    Adhere to the industry best practices for security and efficiency throughout the configuration. The output Terraform config should be in a structured format that is easy to understand and implement.<|end|>\n<|assistant|>\n'

In [ ]:
sampling_params = SamplingParams(temperature=0.7, top_p=0.95, max_tokens=2500, stop=['[/SYS]', "<s>", "</s>","<|end|>"])
outputs= llm.generate(prompt_lst, sampling_params)
terraform_code = [output.outputs[0].text.strip() for output in outputs]

Processed prompts:   0%|          | 0/461 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

WARNING 08-22 00:58:03 scheduler.py:1099] Sequence group 162 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1
WARNING 08-22 00:58:13 scheduler.py:1099] Sequence group 112 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=51
WARNING 08-22 00:58:36 scheduler.py:1099] Sequence group 62 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=101


Processed prompts:   8%|▊         | 39/461 [02:25<17:23,  2.47s/it, est. speed input: 48.66 toks/s, output: 236.59 toks/s]

WARNING 08-22 01:00:28 scheduler.py:1099] Sequence group 66 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=151


Processed prompts:  16%|█▌        | 73/461 [04:14<09:32,  1.48s/it, est. speed input: 52.64 toks/s, output: 317.67 toks/s]

...

Processed prompts:  86%|████████▌ | 397/461 [24:09<03:51,  3.62s/it, est. speed input: 50.92 toks/s, output: 333.16 toks/s]

WARNING 08-22 01:22:11 scheduler.py:1099] Sequence group 426 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1101


Processed prompts:  88%|████████▊ | 407/461 [24:45<02:19,  2.59s/it, est. speed input: 50.93 toks/s, output: 336.57 toks/s]

WARNING 08-22 01:22:56 scheduler.py:1099] Sequence group 441 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1151


Processed prompts: 100%|██████████| 461/461 [28:28<00:00,  3.71s/it, est. speed input: 50.20 toks/s, output: 336.24 toks/s]


In [ ]:
df['microsoft:Phi-3-mini-4k-instruct_code_generation']=terraform_code

In [ ]:
print(df.iloc[4]['microsoft:Phi-3-mini-4k-instruct_code_generation'])

Here's a sample Terraform configuration for creating a unique SSL certificate on Google Cloud Platform (GCP) with a specific description. This configuration uses modules for essential components, adheres to industry best practices for security and efficiency, and is designed to be reusable and easy to maintain.

First, create a module directory for the SSL certificate:

```
.
├── main.tf
├── variables.tf
└── outputs.tf
```

In the `main.tf` file, define the resources required for the SSL certificate:

```hcl
resource "google_certificate" "ssl_certificate" {
  name         = "${local.project_id}-${random_string.suffix.result}"
  domain_name  = "example.com"
  certificate_body = file("${path.module}/certificate.pem")
  private_key   = file("${path.module}/private_key.pem")
  subject_alternative_names = ["*.example.com"]
  description = var.description

  depends_on = [google_compute_address.dns_ip]
}

resource "google_compute_address" "dns_ip" {
  name = "${local.project_id}-${random_str

In [ ]:
df.head()

,Category,Cloud_provider,user_query,Terraform_Code,microsoft:Phi-3-mini-4k-instruct_code_generation
0,GoogleCloudPlatform,GCP,Please generate a Terraform configuration for ...,#\n# Copyright 2020 Google LLC\n#\n# Licensed ...,Here's a basic Terraform configuration for set...
1,immutability-io,AWS,Please generate a Terraform configuration for ...,# DHCP Options\n\n## Set Terraform version con...,Here is a sample Terraform configuration for s...
2,forseti-security,GCP,Please generate a Terraform configuration for ...,/**\n * Copyright 2019 Google LLC\n *\n * Lice...,"```hcl\n\nvariable ""project_id"" {\n\n descrip..."
3,cncf,AWS,Please generate a Terraform configuration for ...,"# resource ""azurerm_network_security_group"" ""c...",Terraform configuration for Google Cloud netwo...
4,terraform-google-modules,GCP,Please generate a Terraform configuration for ...,"resource ""google_compute_ssl_certificate"" ""def...",Here's a sample Terraform configuration for cr...


In [ ]:
df.to_excel("Research_Analysis_Base_data_filtered_new_microsoft:Phi-3-mini-4k-instruct_code_generation.xlsx")

NameError: name 'df' is not defined

In [ ]:
df.shape

In [ ]:
evaluvate_terraform_prompt="""You are an expert Terraform developer specializing in infrastructure as code for cloud environments. I need you to evaluate the given Generated Terraform codes based on a user query and Golden Terraform code to determine whether the generated Terraform codes are correct or not. For each Terraform code, ensure the following:
Correctness: Verify if the code meets the user’s requirements as described in the query.
Best Practices: Check if the code follows best practices for the chosen cloud provider.
Completeness: Ensure all necessary resources and configurations are included.

User query :{}
Golden Terraform Code : {}
Generated terraform Code : {}
The output should contain whether the generated code is  correct or not for the user query and Golden Terraform code , only validate it logically and functionally
example output :
Correct:Yes/No,
Reason : Can be one of criteria mentioned below
1. Security Vulnerabilities
                •              Focus: Look for hard-coded credentials, improper access controls, and insecure resource configurations that could expose the infrastructure to security risks.
2. Incorrect Resource Configuration
                •              Focus: Verify that all resources are correctly configured according to the intended design, including correct instance types, storage configurations, and networking settings.
3. Logical Errors
                •              Focus: Check for errors in the logic, such as incorrect resource dependencies or attributes, which could result in infrastructure that does not behave as expected.
4. Idempotency Issues
                •              Focus: Ensure that the code produces consistent results when applied multiple times, avoiding unintended infrastructure changes that could cause instability.
5. Syntax Errors
                •              Focus: Although less frequent, syntax errors can prevent the Terraform code from being applied. Ensure all syntax is correct and adheres to Terraform's requirements.
6. Resource Deletion or Replacement
                •              Focus: Review the code for any inadvertent deletion or replacement of critical resources, which could lead to data loss or significant downtime.
7. Version Compatibility Issues
                •              Focus: Ensure that the code is compatible with the specific Terraform and provider versions in use, avoiding deprecated features or version-specific bugs.
8. Misuse of Modules
                •              Focus: Check that modules are used correctly, with appropriate input and output variables, ensuring that the infrastructure is modular and maintainable.
9. Networking Configuration Issues
                •              Focus: Pay attention to networking components, ensuring that VPCs, subnets, and security groups are correctly configured to avoid network isolation or exposure to threats.
10. Performance and Efficiency
                •              Focus: Evaluate the efficiency of the code, looking for any unnecessary complexity or suboptimal resource usage that could lead to increased costs or poor performance.
"""

In [ ]:
df=pd.read_excel('/content/Research_Analysis_Base_data_filtered_new_microsoft_Phi-3-mini-4k-instruct_code_generation.xlsx')
del df['Unnamed: 0']
print(len(df))

461


In [ ]:
# Define the function to determine the cloud provider
def evaluate_terraform(inp):
    golden=inp['Terraform_Code']
    user_query=inp['user_query']
    generated=inp['microsoft:Phi-3-mini-4k-instruct_code_generation']
    updated_prompt=evaluvate_terraform_prompt.format(user_query,golden,generated)
    res=chatGPT4oresponse(updated_prompt)
    return (res)

# Apply the function only where Cloud_provider is 'NVF'
df['GPT4_Validation_Result'] = df.apply(evaluate_terraform,axis=1)

Streaming output truncated to the last 5000 lines.
2. **Logical Errors**:
   - The generated code does not correctly associate the route table with the subnet. The Golden Terraform code includes an `aws_route_table_association` resource to associate the route table with the subnet, which is missing in the generated code.
   - The generated code does not use the `var.vpc_id` variable for the VPC ID in the `aws_subnet`, `aws_security_group`, and `aws_route_table` resources, which is required to ensure the resources are created within the specified VPC.

3. **Misuse of Modules**:
   - The generated code uses modules incorrectly. The `module "vpc"`, `module "route_table"`, and `module "security_group"` all source from `./modules/vpc`, which is not modular and does not align with the Golden Terraform code structure. Each module should have its own source directory and specific input/output variables.

4. **Networking Configuration Issues**:
   - The generated code does not correctly configu

KeyboardInterrupt: 

In [ ]:
df.to_excel("Research_Analysis_Base_data_filtered_new_microsoft:Phi-3-mini-4k-instruct_code_generation_validated.xlsx")